## Implementação Prática
A _Cia Ótica Gallão_ pretende utilizar os canteiros do mapa rodoviário abaixo para passar o novo cabeamento de fibra ótica entre essas cidades, com base na origem e destino que você informar:

<div align="center">
    <img src="./src/img/mapaCidades.png" alt="Mapa Rodoviário" />
</div> 

In [6]:
# Importação das bibliotecas necessárias para execução do algoritmo
from collections import deque
# Importação das bibliotecas necessárias para exibição do diagrama
import base64 
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Grafo do mapa rodoviário
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba',30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

# Associação entre os nomes das cidades e os vértices do grafo (para exibição)
nome_das_cidades = {'Piracicaba': 'Piracicaba', 'Americana': 'Americana', 'Sumaré': 'Sumare', 
                 'Campinas': 'Campinas', 'Monte Mor': 'MonteMor', 'Indaiatuba': 'Indaiatuba',
                 'Salto': 'Salto', 'Capivari': 'Capivari', 'Tietê': 'Tiete', 'Paulínea': 'Paulinea',
                 'Porto Feliz': 'PortoFeliz', 'Tatuí': 'Tatui', 'Itú': 'Itu', 'Sorocaba': 'Sorocaba',
                 'Boituva': 'Boituva'}

# Algoritmo de Kruskal
def algoritmoKruskal(grafo):
    arestas = []

    # Cria uma lista de arestas com os pesos
    for origem in grafo:
        for destino, peso in grafo[origem]:
            arestas.append((origem, destino, peso))
    # Ordena as arestas pelo peso        
    arestas.sort(key=lambda x: x[2])  
    
    # Inicializa a floresta
    floresta = {cidade: [cidade] for cidade in grafo}       # Floresta de árvores

    # Inicializa a árvore mínima
    arvore = set()
    
    # Percorre as arestas em ordem crescente de peso
    for origem, destino, peso in arestas:
        if floresta[origem] != floresta[destino]:           # Verifica se as cidades pertencem a árvores diferentes
            arvore.add((origem, destino, peso))             # Adiciona a aresta na árvore geradora
            nova_floresta = floresta[origem] + floresta[destino]

            for cidade in nova_floresta:
                floresta[cidade] = nova_floresta            # Atualiza a floresta para conectar as cidades
    
    return arvore

# Interação com o usuário
origem = input("Informe a cidade em que pretende iniciar a passagem dos cabos: ")
destino = input("Informe a cidade em que pretende terminar: ")

# Execução do algoritmo
arvore = algoritmoKruskal(grafo, origem, destino)

# Exibição do resultado
print("A árvore geradora mínima é:")
for origem, destino, peso in arvore:
    print(f"{nome_das_cidades[origem]} - {nome_das_cidades[destino]}: {peso} km")


A árvore geradora mínima é:
Tatui - Boituva: 17 km
PortoFeliz - Boituva: 12 km
Sumare - Campinas: 23 km
Indaiatuba - Salto: 20 km
Tiete - Tatui: 25 km
Itu - Sorocaba: 8 km
Americana - Sumare: 18 km
PortoFeliz - Itu: 12 km
Piracicaba - Americana: 30 km
Americana - Paulinea: 22 km
Capivari - MonteMor: 15 km
Indaiatuba - Campinas: 20 km
Salto - Itu: 10 km
MonteMor - Campinas: 22 km


In [3]:
class Grafo:
    def __init__(self, grafo):
        self.grafo = grafo
        self.arestas = []
        for cidade, conexoes in grafo.items():
            for conexao in conexoes:
                cidade_destino, peso = conexao
                self.arestas.append((peso, cidade, cidade_destino))
        self.arestas.sort()  # Ordena as arestas pelo peso (primeiro elemento da tupla)

    def encontra(self, pai, vertice):
        if pai[vertice] == vertice:
            return vertice
        return self.encontra(pai, pai[vertice])

    def uniao(self, pai, rank, vertice1, vertice2):
        raiz1 = self.encontra(pai, vertice1)
        raiz2 = self.encontra(pai, vertice2)

        if rank[raiz1] < rank[raiz2]:
            pai[raiz1] = raiz2
        elif rank[raiz1] > rank[raiz2]:
            pai[raiz2] = raiz1
        else:
            pai[raiz2] = raiz1
            rank[raiz1] += 1

    def kruskal(self):
        arvore = []  # Armazenará as arestas da árvore mínima
        pai = {}
        rank = {}

        # Inicialização: cada cidade é seu próprio pai
        for cidade in self.grafo.keys():
            pai[cidade] = cidade
            rank[cidade] = 0

        # Percorre todas as arestas e aplica a lógica de Kruskal
        for aresta in self.arestas:
            peso, cidade1, cidade2 = aresta

            raiz1 = self.encontra(pai, cidade1)
            raiz2 = self.encontra(pai, cidade2)

            if raiz1 != raiz2:
                arvore.append((cidade1, cidade2, peso))
                self.uniao(pai, rank, raiz1, raiz2)

        return arvore

def exibir_arvore_minima(arvore):
    print("Árvore de peso mínimo completa:")
    for cidade1, cidade2, peso in arvore:
        print(f"{cidade1} -- {cidade2} (Distância: {peso})")

def construir_grafo_arvore(arvore):
    grafo = {}
    for cidade1, cidade2, peso in arvore:
        if cidade1 not in grafo:
            grafo[cidade1] = []
        if cidade2 not in grafo:
            grafo[cidade2] = []
        grafo[cidade1].append((cidade2, peso))
        grafo[cidade2].append((cidade1, peso))
    return grafo

def dfs_caminho(grafo, origem, destino, visitado=None, caminho=None, distancia=0):
    if visitado is None:
        visitado = set()
    if caminho is None:
        caminho = []

    visitado.add(origem)
    caminho.append((origem, distancia))

    if origem == destino:
        return caminho

    for vizinho, peso in grafo.get(origem, []):
        if vizinho not in visitado:
            resultado = dfs_caminho(grafo, vizinho, destino, visitado, caminho, peso)
            if resultado:
                return resultado

    caminho.pop()
    return None

def calcular_distancia(caminho):
    distancia_total = 0
    for i in range(1, len(caminho)):
        distancia_total += caminho[i][1]
    return distancia_total

# Mapa rodoviário
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba',30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

# Recebe entrada do usuário
origem = input("Digite a cidade de origem: ")
destino = input("Digite a cidade de destino: ")

# Cria o objeto Grafo
g = Grafo(grafo)

# Executa o algoritmo de Kruskal para obter a árvore mínima
arvore_minima = g.kruskal()

# Exibe a árvore mínima completa
exibir_arvore_minima(arvore_minima)

# Constrói o grafo da árvore mínima
grafo_arvore = construir_grafo_arvore(arvore_minima)

# Encontra o caminho da origem até o destino na árvore mínima
caminho = dfs_caminho(grafo_arvore, origem, destino)

if caminho:
    print("\nCaminho da origem até o destino:")
    for cidade, distancia in caminho:
        if distancia == 0:
            print(cidade, end="")
        else:
            print(f" --({distancia})--> {cidade}", end="")
    print()

    # Calcula a distância total
    distancia_total = calcular_distancia(caminho)
    print(f"\nDistância total percorrida: {distancia_total} km")
else:
    print(f"Não foi encontrado um caminho de {origem} até {destino}.")


Árvore de peso mínimo completa:
Itú -- Sorocaba (Distância: 8)
Itú -- Salto (Distância: 10)
Boituva -- Porto Feliz (Distância: 12)
Itú -- Porto Feliz (Distância: 12)
Capivari -- Monte Mor (Distância: 15)
Boituva -- Tatuí (Distância: 17)
Americana -- Sumaré (Distância: 18)
Campinas -- Indaiatuba (Distância: 20)
Indaiatuba -- Salto (Distância: 20)
Americana -- Paulínea (Distância: 22)
Campinas -- Monte Mor (Distância: 22)
Campinas -- Sumaré (Distância: 23)
Tatuí -- Tietê (Distância: 25)
Americana -- Piracicaba (Distância: 30)

Caminho da origem até o destino:
Boituva --(12)--> Porto Feliz --(12)--> Itú --(10)--> Salto --(20)--> Indaiatuba --(20)--> Campinas --(22)--> Monte Mor --(15)--> Capivari

Distância total percorrida: 111 km


In [7]:
# Grafo do mapa rodoviário
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba', 30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

# Associação entre os nomes das cidades e os vértices do grafo (para exibição)
nome_das_cidades = {
    'Piracicaba': 'Piracicaba', 'Americana': 'Americana', 'Sumaré': 'Sumare',
    'Campinas': 'Campinas', 'Monte Mor': 'MonteMor', 'Indaiatuba': 'Indaiatuba',
    'Salto': 'Salto', 'Capivari': 'Capivari', 'Tietê': 'Tiete', 'Paulínea': 'Paulinea',
    'Porto Feliz': 'PortoFeliz', 'Tatuí': 'Tatui', 'Itú': 'Itu', 'Sorocaba': 'Sorocaba',
    'Boituva': 'Boituva'
}

# Algoritmo de busca em profundidade (DFS) para encontrar o caminho
def busca_em_profundidade(grafo, origem, destino, caminho=[]):
    caminho = caminho + [origem]
    if origem == destino:
        return caminho
    if origem not in grafo:
        return None
    for (cidade, distancia) in grafo[origem]:
        if cidade not in caminho:
            novo_caminho = busca_em_profundidade(grafo, cidade, destino, caminho)
            if novo_caminho:
                return novo_caminho
    return None

# Geração do flowchart em formato Mermaid
def gerar_mermaid_flowchart(caminho):
    flowchart = "flowchart TD\n"
    for i in range(len(caminho) - 1):
        flowchart += f"    {nome_das_cidades[caminho[i]]} --> {nome_das_cidades[caminho[i + 1]]}\n"
    return flowchart

# Interação com o usuário
origem = input("Informe a cidade em que pretende iniciar a viagem: ")
destino = input("Informe a cidade em que pretende terminar: ")

# Execução da busca em profundidade
caminho = busca_em_profundidade(grafo, origem, destino)

# Verificação do resultado
if caminho:
    print("Caminho encontrado:")
    for cidade in caminho:
        print(f"- {nome_das_cidades[cidade]}")

    # Gerar o Mermaid flowchart
    flowchart_mermaid = gerar_mermaid_flowchart(caminho)
    print("\nFlowchart Mermaid:")
    print(flowchart_mermaid)
else:
    print("Nenhum caminho encontrado entre as cidades informadas.")


Caminho encontrado:
- Boituva
- PortoFeliz
- Itu

Flowchart Mermaid:
flowchart TD
    Boituva --> PortoFeliz
    PortoFeliz --> Itu



In [8]:
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba',30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

nome_das_cidades = {
    'Piracicaba': 'Piracicaba', 'Americana': 'Americana', 'Sumaré': 'Sumare', 
    'Campinas': 'Campinas', 'Monte Mor': 'MonteMor', 'Indaiatuba': 'Indaiatuba',
    'Salto': 'Salto', 'Capivari': 'Capivari', 'Tietê': 'Tiete', 'Paulínea': 'Paulinea',
    'Porto Feliz': 'PortoFeliz', 'Tatuí': 'Tatui', 'Itú': 'Itu', 'Sorocaba': 'Sorocaba',
    'Boituva': 'Boituva'
}

def algoritmoKruskal(grafo):
    arestas = []

    # Cria uma lista de arestas com os pesos
    for origem in grafo:
        for destino, peso in grafo[origem]:
            arestas.append((origem, destino, peso))
    arestas.sort(key=lambda x: x[2])  # Ordena as arestas pelo peso
    
    floresta = {cidade: [cidade] for cidade in grafo}  # Floresta de árvores
    arvore = set()  # Árvore geradora mínima
    
    for origem, destino, peso in arestas:
        if floresta[origem] != floresta[destino]:  # Verifica se as cidades pertencem a árvores diferentes
            arvore.add((origem, destino, peso))  # Adiciona a aresta na árvore geradora
            nova_floresta = floresta[origem] + floresta[destino]
            for cidade in nova_floresta:
                floresta[cidade] = nova_floresta  # Atualiza a floresta para as cidades conectadas
    
    return arvore

# Interação com o usuário
origem = input("Informe a cidade em que pretende iniciar a passagem dos cabos: ")
destino = input("Informe a cidade em que pretende terminar: ")

# Executa o algoritmo de Kruskal para encontrar a árvore mínima
arvore = algoritmoKruskal(grafo)

# Gera o gráfico em formato Mermaid
def gerar_mermaid_diagrama(arvore):
    print("```mermaid")
    print("graph TD")
    
    for origem, destino, peso in arvore:
        origem_nome = nome_das_cidades[origem]
        destino_nome = nome_das_cidades[destino]
        print(f"{origem_nome} -->|{peso} km| {destino_nome}")
    
    print("```")

# Exibição do resultado
print("A árvore geradora mínima em formato Mermaid é:")
gerar_mermaid_diagrama(arvore)


A árvore geradora mínima em formato Mermaid é:
```mermaid
graph TD
Tatui -->|17 km| Boituva
PortoFeliz -->|12 km| Boituva
Sumare -->|23 km| Campinas
Indaiatuba -->|20 km| Salto
Tiete -->|25 km| Tatui
Itu -->|8 km| Sorocaba
Americana -->|18 km| Sumare
PortoFeliz -->|12 km| Itu
Piracicaba -->|30 km| Americana
Americana -->|22 km| Paulinea
Capivari -->|15 km| MonteMor
Indaiatuba -->|20 km| Campinas
Salto -->|10 km| Itu
MonteMor -->|22 km| Campinas
```


In [9]:
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba',30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

nome_das_cidades = {
    'Piracicaba': 'Piracicaba', 'Americana': 'Americana', 'Sumaré': 'Sumare', 
    'Campinas': 'Campinas', 'Monte Mor': 'MonteMor', 'Indaiatuba': 'Indaiatuba',
    'Salto': 'Salto', 'Capivari': 'Capivari', 'Tietê': 'Tiete', 'Paulínea': 'Paulinea',
    'Porto Feliz': 'PortoFeliz', 'Tatuí': 'Tatui', 'Itú': 'Itu', 'Sorocaba': 'Sorocaba',
    'Boituva': 'Boituva'
}

def algoritmoKruskal(grafo):
    arestas = []

    # Cria uma lista de arestas com os pesos
    for origem in grafo:
        for destino, peso in grafo[origem]:
            arestas.append((origem, destino, peso))
    arestas.sort(key=lambda x: x[2])  # Ordena as arestas pelo peso
    
    floresta = {cidade: [cidade] for cidade in grafo}  # Floresta de árvores
    arvore = set()  # Árvore geradora mínima
    
    for origem, destino, peso in arestas:
        if floresta[origem] != floresta[destino]:  # Verifica se as cidades pertencem a árvores diferentes
            arvore.add((origem, destino, peso))  # Adiciona a aresta na árvore geradora
            nova_floresta = floresta[origem] + floresta[destino]
            for cidade in nova_floresta:
                floresta[cidade] = nova_floresta  # Atualiza a floresta para as cidades conectadas
    
    return arvore

# Interação com o usuário
origem = input("Informe a cidade em que pretende iniciar a passagem dos cabos: ")
destino = input("Informe a cidade em que pretende terminar: ")

# Executa o algoritmo de Kruskal para encontrar a árvore mínima
arvore = algoritmoKruskal(grafo)

# Gera o gráfico em formato Mermaid
def gerar_mermaid_diagrama(arvore):
    print("```mermaid")
    print("graph TD")
    
    for origem, destino, peso in arvore:
        origem_nome = nome_das_cidades[origem]
        destino_nome = nome_das_cidades[destino]
        print(f"{origem_nome} -->|{peso} km| {destino_nome}")
    
    print("```")

# Exibição do resultado
print("A árvore geradora mínima em formato Mermaid é:")
gerar_mermaid_diagrama(arvore)


A árvore geradora mínima em formato Mermaid é:
```mermaid
graph TD
Tatui -->|17 km| Boituva
PortoFeliz -->|12 km| Boituva
Sumare -->|23 km| Campinas
Indaiatuba -->|20 km| Salto
Tiete -->|25 km| Tatui
Itu -->|8 km| Sorocaba
Americana -->|18 km| Sumare
PortoFeliz -->|12 km| Itu
Piracicaba -->|30 km| Americana
Americana -->|22 km| Paulinea
Capivari -->|15 km| MonteMor
Indaiatuba -->|20 km| Campinas
Salto -->|10 km| Itu
MonteMor -->|22 km| Campinas
```
